## 0. Simulate some data and fit emulator

In [1]:
import torch

from autoemulate.experimental.simulations.projectile import ProjectileMultioutput
from autoemulate.experimental.emulators.gaussian_process.exact import (
    GaussianProcessExact,
)

In [2]:
sim = ProjectileMultioutput()
x = sim.sample_inputs(10)
y = sim.forward_batch(x)

Running simulations: 100%|██████████| 10/10 [00:00<00:00, 547.37it/s]

Successfully completed 10/10 simulations (100.0%)


In [3]:
# check range of training outputs
torch.min(y, dim=0).values, torch.max(y, dim=0).values

(tensor([1.0018, 1.5183], dtype=torch.float64),
 tensor([52476.1038,   599.9588], dtype=torch.float64))

In [4]:
gp = GaussianProcessExact(x, y)
gp.fit(x, y)

## 1. Simple HMC example.

In [5]:
from autoemulate.experimental.calibration.hmc import HMCCalibrator

Lets start with an observation inside the training range, we should be able to recover the input parameters.

In [6]:
# given the ranges, we can actually just them to the same thing
observations = {name: 100 for name in sim.output_names}

In [7]:
hmc = HMCCalibrator(gp, sim.parameters_range, observations, 1.0)

In [8]:
mcmc = hmc.run_mcmc(warmup_steps=10, num_samples=100)

Sample: 100%|██████████| 110/110 [00:00, 348.17it/s, step size=4.36e-02, acc. prob=0.003]


The returned Pyro MCMC object has methods for accessing the generated samples (`mcmc.get_samples()`) or, as shown below, to get just their summary statistics.

In [9]:
mcmc.summary()


                mean       std    median      5.0%     95.0%     n_eff     r_hat
         c     -1.29      0.04     -1.25     -1.34     -1.25      2.64      6.99
        v0    652.24      0.13    652.30    652.06    652.40      5.86      1.25

Number of divergences: 100


## 2. Combining this with sensitivity analysis and history matching.

The `HMCCalibrator` object has an option to provide a list of parameters to calibrate. These can be the result of `SensitivityAnalysis`, or just a list provided by the user.

Similarly, the user provides parameter ranges from which to sample or set the parameter values. This can be simply the range of the simulator or one can use `HistoryMatching` to reduce the parameter range and pass that to the `HMCCalibrator` instead. 

Below we demonstrate how to do both.

In [10]:
from autoemulate.experimental.sensitivity_analysis import SensitivityAnalysis
from autoemulate.experimental.calibration.history_matching import HistoryMatching

1. Run sensitivity analysis and get top N parameters (here we just get the top 1).

In [11]:
problem = {
        "num_vars": 2,
        "names": ["c", "v0"],
        "bounds": [(-5.0, 1.0), (0.0, 1000.0)],
    }
sa = SensitivityAnalysis(gp, problem=problem)
df = sa.run("sobol")

/Users/rjersakova/Documents/Projects/autoemulate/.venv/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
/Users/rjersakova/Documents/Projects/autoemulate/.venv/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


In [12]:
top_param = sa.top_n_params(df, 1)
top_param

['v0']

2. Run history matching and generate new parameter bounds from NROY samples.

In [13]:
# start with some GP predictions
x_new = sim.sample_inputs(10)
output = gp.predict(torch.tensor(x_new, dtype=torch.float32))
pred_means, pred_vars = (
    output.mean.float().detach(),
    output.variance.float().detach(),
)

/var/folders/bj/kdwy1bhj3h728lr5xdj19yd40000gr/T/ipykernel_69876/680583242.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = gp.predict(torch.tensor(x_new, dtype=torch.float32))


In [20]:
# generate NROY samples
hm = HistoryMatching(
    observations={"v0": (10, 5), "c": (10 ,5)},
    threshold=3.0
)
implausability = hm.calculate_implausibility(pred_means, pred_vars)
nroy_samples = hm.get_nroy(implausability, x_new)

In [22]:
# get new param bounds
nroy_param_range = hm.generate_param_bounds(nroy_samples, param_names = ["v0", "c"])
nroy_param_range

{'v0': [tensor(-5.0463), tensor(1.1488)],
 'c': [tensor(-32.9765), tensor(899.2466)]}

3. Pass results to the HMCCalibrator object.

In [23]:
hmc = HMCCalibrator(
    gp, 
    nroy_param_range, 
    observations, 
    1.0,
    top_param
    )

In [25]:
mcmc = hmc.run_mcmc()

Sample: 100%|██████████| 1500/1500 [00:04, 318.37it/s, step size=1.07e+00, acc. prob=0.882]


In [26]:
mcmc.summary()


                mean       std    median      5.0%     95.0%     n_eff     r_hat
        v0     -1.96      1.76     -1.82     -5.01      0.38    409.20      1.00

Number of divergences: 0
